In [275]:
# 가상환경 생성 (Python 3.10 또는 3.11)

# ----- terminal에서 실행 -----
# py -3.10 -m venv agent-env                                  <- venv 생성
# Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass  <- PS 보안 비활성화
# .\Task_recommend\use_agent\agent-env\Scripts\activate       <- venv 활성화

In [276]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [327]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from jinja2 import Template

from dotenv import load_dotenv
import json
import os
import re
import json
import random

In [328]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [329]:
JSON_FILE_PATH = "..\\use_openai\\data\\id_added_dummy_data.json"
RCMD_PROMPT_PATH = "..\\prompts\\agent_recommend.md"
FEATURE_ID_PATH = "featureId.txt"

In [330]:
# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()
    
template = Template(RCMD_PROMPT)  # template 객체 생성

# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(type(data))  # test

# Read featureId keys
with open(FEATURE_ID_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    feature_id_list = [line.strip() for line in lines]
    # print(feature_id)  # test

<class 'list'>


In [331]:
FEATURE_ID = random.choice(feature_id_list)
print(FEATURE_ID, type(FEATURE_ID))

36 <class 'str'>


In [332]:
# featureId로 해당 category, feature 이하의 actions만 추출출
FEATURE_ID = random.choice(feature_id_list)

def extract_workspace_view(input_data, id):
    for item in input_data:
        for category in item.get("recommendedCategories", []):
            for feature in category.get("features", []):
                if feature.get("featureId") == int(id):
                    return {
                        "categoryId": category["categoryId"],
                        "name": category["name"],
                        "features": [feature]
                    }
    return None

result = extract_workspace_view(data, FEATURE_ID)
# print(json.dumps(result, ensure_ascii=False, indent=2))

if result:
    print(json.dumps(result, ensure_ascii=False, indent=2))
else:
    print("해당 featureId를 찾을 수 없습니다.")

{
  "categoryId": 4,
  "name": "게시판 기능",
  "features": [
    {
      "featureId": 24,
      "name": "관리 기능",
      "actions": [
        {
          "name": "스팸 필터링",
          "importance": 2,
          "startDate": "2025-06-16T14:25:34.094434",
          "endDate": "2025-06-17T14:25:34.094434"
        },
        {
          "name": "게시판 설정",
          "importance": 4,
          "startDate": "2025-06-20T14:25:34.094437",
          "endDate": "2025-07-01T14:25:34.094437"
        },
        {
          "name": "게시글 승인",
          "importance": 1,
          "startDate": "2025-06-16T14:25:34.094440",
          "endDate": "2025-06-28T14:25:34.094440"
        }
      ]
    }
  ]
}


In [326]:
workspace_data = extract_workspace_view(data, FEATURE_ID)
wrapped_data = {
    "workspaceId": 1,
    "recommendedCategories": [workspace_data]
}
formatted_input = template.render(wrapped_data=wrapped_data)  # 문자열 생성. template 내 변수들을 실제값으로 치환

print(formatted_input)  # test

Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:

{
  "workspaceId": "작업 중인 workspaceId",
  "recommendedCategories": [
    {
      "categoryId": "작업 중인 categoryId",
      "name": "카테고리 이름",
      "startDate": "YYYY-MM-DDTHH:MM:SS",
      "endDate": "YYYY-MM-DDTHH:MM:SS",
      "importance": 정수,
      "features": [
        {
          "featureId": "작업 중인 featureId",
          "name": "기능 이름",
          "startDate": "YYYY-MM-DDTHH:MM:SS",
          "endDate": "YYYY-MM-DDTHH:MM:SS",
          "importance": 정수,
          "actions": [
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS",
              "endDate": "YYYY-MM-DDTHH:MM:SS",
              "importance": 정수
            },
            {
              "name": "추천 action",
              "startDate": "YYYY-MM-DDTHH:MM:SS

In [333]:
# OpenAI 객체 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,
    max_tokens=1024,
    openai_api_key=openai_api_key
)

In [299]:
# 변수{} 삽입 가능한 템플릿 생성
# prompt = PromptTemplate.from_template("""
# 다음은 '{featureName}' 기능의 기존 작업들입니다:

# {existingActions}

# 이 작업의 다음 단계로서, 자연스럽게 이어질 수 있는 작업 3개를 추천하세요.
# 각 작업은 다음 항목을 포함해야 합니다:

# - name: 구체적 작업명
# - startDate: YYYY-MM-DDTHH:MM:SS 형식
# - endDate: YYYY-MM-DDTHH:MM:SS 형식
# - importance: 1~5 정수(값이 클수록 중요한 작업업)

# 형식은 반드시 JSON 배열 형태로 출력하세요.
# 예시:
# ```json
# [
#   {
#     "name": "...",
#     "startDate": "...",
#     "endDate": "...",
#     "importance": 3
#   },
#   ...
# ]
# ```
# """)

prompt = PromptTemplate(
    input_variables=["featureNname", "existingActions"],
    template="""
    다음은 '{featureName}' 기능의 기존 작업들입니다:

    {existingActions}

    이 작업의 다음 단계로서, 자연스럽게 이어질 수 있는 작업 3개를 추천하세요.
    각 작업은 다음 항목을 포함해야 합니다:

    - name: 구체적 작업명
    - startDate: YYYY-MM-DDTHH:MM:SS 형식
    - endDate: YYYY-MM-DDTHH:MM:SS 형식
    - importance: 1~5 정수(값이 클수록 중요한 작업업)

    형식은 반드시 JSON 배열 형태로 출력하세요.
    예시:
    ```json
    [
      {
        "name": "...",
        "startDate": "...",
        "endDate": "...",
        "importance": 3
      },
      ...
    ]
    ```
    """
)

# llm(OpenAI) <-> Langchain 연결
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [349]:
##### test #####
search = DuckDuckGoSearchRun()
conv_data = json.dumps(wrapped_data, ensure_ascii=False)  # dict -> str
pattern = r'({[\s\S]*})'
match = re.search(pattern, conv_data, re.DOTALL)
json_block = match.group(0)
print(type(json_block))
print(json_block)


<class 'str'>
{"workspaceId": 1, "recommendedCategories": [{"categoryId": 5, "name": "알림 시스템", "features": [{"featureId": 21, "name": "실시간 알림", "actions": [{"name": "푸시 알림", "importance": 4, "startDate": "2025-06-17T14:25:34.095319", "endDate": "2025-06-18T14:25:34.095319"}, {"name": "SMS 알림", "importance": 4, "startDate": "2025-06-20T14:25:34.095321", "endDate": "2025-06-28T14:25:34.095321"}]}]}]}


In [375]:
# LLM으로 작업 추천 받는 함수
def generate_next_actions(input_dict: dict) -> str:
    return llm_chain.run(input_dict)

# 웹 검색 툴 정의
search = DuckDuckGoSearchRun()

# 전체 JSON을 받아 feature별 추천 작업을 생성하는 메인 함수
def parse_and_generate_all(input_str: dict) -> str:

    # conv_data = json.dumps(input_str, ensure_ascii=False)  # dict -> str
    # pattern = r'({[\s\S]*?"categoryId"\s*:\s*\d+[\s\S]*?})'
    # match = re.search(pattern, conv_data, re.DOTALL)
    data = input_str
    if not match:
        raise ValueError("** Can not find JSON blocks **")

    json_block = match.group(0)
    data = json.loads(json_block)
    # data = input_str

    for category in data.get("recommendedCategories", []):  # recommendedCategories 가져오기
        for feature in category.get("features", []):        # recommendedCategories 내부의 feature에서
            # llm_input = {                                   # features, actions 가져오기
            #     "featureName": feature["name"],
            #     "existingActions": [a["name"] for a in feature.get("actions", [])]
            # }

            # 웹 검색
            try:
                search_query = f"{feature['name']} 기능 구현 방법"
                search_result = search.run(search_query)
            except Exception as e:
                search_result = f"(웹 검색 실패: {e})"

            # LLM 입력 구성 (웹검색 결과 포함)
            llm_input = {
                "featureName": feature["name"],
                "existingActions": [a["name"] for a in feature.get("actions", [])],
                "webSearch": search_result[:500]  # 길이 제한
            }

            try:
                generated_json_str = generate_next_actions(llm_input)  # 추천 actions 생성
                generated_actions = json.loads(generated_json_str)     # 추천 actions를 json형식의 python 객체로 load
                feature["actions"].extend(generated_actions)
            except Exception as e:
                feature["actions"].append({
                    "name": f"작업 생성 실패: {e}",
                    "startDate": "2025-01-01T00:00:00",
                    "endDate": "2025-01-01T23:59:59",
                    "importance": 0
                })

    return json.dumps(data, ensure_ascii=False, indent=2)


In [376]:
type(wrapped_data)
# wrapped_data

dict

In [377]:
parse_and_generate_all(wrapped_data)

'{\n  "workspaceId": 1,\n  "recommendedCategories": [\n    {\n      "categoryId": 5,\n      "name": "알림 시스템",\n      "features": [\n        {\n          "featureId": 21,\n          "name": "실시간 알림",\n          "actions": [\n            {\n              "name": "푸시 알림",\n              "importance": 4,\n              "startDate": "2025-06-17T14:25:34.095319",\n              "endDate": "2025-06-18T14:25:34.095319"\n            },\n            {\n              "name": "SMS 알림",\n              "importance": 4,\n              "startDate": "2025-06-20T14:25:34.095321",\n              "endDate": "2025-06-28T14:25:34.095321"\n            },\n            {\n              "name": "작업 생성 실패: Missing some input keys: {\'\\\\n        \\"name\\"\'}",\n              "startDate": "2025-01-01T00:00:00",\n              "endDate": "2025-01-01T23:59:59",\n              "importance": 0\n            }\n          ]\n        }\n      ]\n    }\n  ]\n}'

In [378]:
# 툴 정의
parse_tool = Tool(
    name="ParseWorkflow",
    func=parse_and_generate_all,
    description="JSON 작업 흐름에 대해 feature별로 다음 추천 작업(actions) 3개를 추가합니다."
)

final_tool = Tool(
    name="Final Answer",
    func=lambda x: x,  # 결과를 그대로 반환
    description="최종 JSON 결과를 반환합니다."
)

In [379]:
# Agent
custom_prompt = formatted_input

agent_executor = initialize_agent(
    tools=[parse_tool, final_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    prompt=custom_prompt
)

In [380]:
# type(wrapped_data)

In [381]:
# # prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(wrapped_data, ensure_ascii=False))
# prompt_filled = RCMD_PROMPT.render(input=json.load(wrapped_data, ensure_ascii=False))

# # type(prompt_filled)
# prompt_filled

In [382]:
prompt_filled = RCMD_PROMPT.replace("{input}", json.dumps(wrapped_data, ensure_ascii=False))

response = agent_executor.invoke({"input": prompt_filled})

# response = agent_executor.invoke({
#     "input": "다음 JSON을 분석해주세요.",
#     "data": prompt_filled  # JSON 객체 그대로
# })

print(response)



> Entering new AgentExecutor chain...
{
  "workspaceId": "1",
  "recommendedCategories": [
    {
      "categoryId": "5",
      "name": "알림 시스템",
      "features": [
        {
          "featureId": "21",
          "name": "실시간 알림",
          "startDate": "2025-06-17T14:25:34.095319",
          "endDate": "2025-06-18T14:25:34.095319",
          "importance": 4,
          "actions": [
            {
              "name": "푸시 알림 설정",
              "startDate": "2025-06-19T14:25:34.095319",
              "endDate": "2025-06-19T15:25:34.095319",
              "importance": 5
            },
            {
              "name": "SMS 알림 테스트",
              "startDate": "2025-06-21T14:25:34.095321",
              "endDate": "2025-06-21T15:25:34.095321",
              "importance": 4
            },
            {
              "name": "알림 로그 분석",
              "startDate": "2025-06-22T14:25:34.095321",
              "endDate": "2025-06-22T15:25:34.095321",
              "importance": 3
         

In [384]:
response

{'input': 'Question: 아래의 JSON 작업 흐름을 바탕으로 각 feature에 대해 이어질 작업 3개씩 추천해주세요. 추천 작업은 feature의 actions 배열에 포함되어야 하며, name, startDate, endDate, importance가 반드시 포함되어야 합니다. 결과는 반드시 JSON 형식으로 출력하고, 추가 설명 없이 JSON만 출력해야 합니다. 출력 예시는 아래 형식을 따릅니다:\n\n{\n  "workspaceId": "작업 중인 workspaceId",\n  "recommendedCategories": [\n    {\n      "categoryId": "작업 중인 categoryId",\n      "name": "카테고리 이름",\n      "startDate": "YYYY-MM-DDTHH:MM:SS",\n      "endDate": "YYYY-MM-DDTHH:MM:SS",\n      "importance": 정수,\n      "features": [\n        {\n          "featureId": "작업 중인 featureId",\n          "name": "기능 이름",\n          "startDate": "YYYY-MM-DDTHH:MM:SS",\n          "endDate": "YYYY-MM-DDTHH:MM:SS",\n          "importance": 정수,\n          "actions": [\n            {\n              "name": "추천 action",\n              "startDate": "YYYY-MM-DDTHH:MM:SS",\n              "endDate": "YYYY-MM-DDTHH:MM:SS",\n              "importance": 정수\n            },\n            {\n              "name": "추천 action",\n         